In [20]:
import numpy as np
import torch
import random
from torch.utils import data
from d2l import torch as d2l
import time

In [23]:
# 计时器
class Timer:  #@save
    """记录多次运行时间"""
    def __init__(self):
        self.times = []
        self.start()

    def start(self):
        """启动计时器"""
        self.tik = time.time()

    def stop(self):
        """停止计时器并将时间记录在列表中"""
        self.times.append(time.time() - self.tik)
        return self.times[-1]

    def avg(self):
        """返回平均时间"""
        return sum(self.times) / len(self.times)

    def sum(self):
        """返回时间总和"""
        return sum(self.times)

    def cumsum(self):
        """返回累计时间"""
        return np.array(self.times).cumsum().tolist()
    
n = 10000
a = torch.ones([n])
b = torch.ones([n])
c = torch.zeros(n)
timer = Timer()
for i in range(n):
    c[i] = a[i] + b[i]
print(f'{timer.stop():.5f} sec')

timer.start()
d = a + b
print(f'{timer.stop():.5f} sec')


0.14262 sec
0.00000 sec


In [7]:
## 生成数据集

def synthetic_data(w,b,num_examples):
    x = torch.normal(0,1,(num_examples,len(w)))
    y = torch.matmul(x,w) + b
    y += torch.normal(0,0.01,y.shape)
    return x,y.reshape(-1,1)




true_w = torch.tensor([2, -3.4])
true_b = 4.2
features, labels = synthetic_data(true_w, true_b, 1000)



features, labels

(tensor([[-1.0458e-03,  1.0712e+00],
         [-5.1186e-01, -4.4212e-01],
         [-1.9893e+00, -1.8686e-01],
         ...,
         [-2.4901e-01, -1.3419e-02],
         [ 5.8908e-01, -9.1967e-01],
         [ 1.2828e+00,  8.6837e-01]]),
 tensor([[ 5.6717e-01],
         [ 4.6765e+00],
         [ 8.5276e-01],
         [ 9.8281e+00],
         [-1.0077e+00],
         [ 2.9404e+00],
         [-2.7192e+00],
         [ 8.2972e+00],
         [ 4.1820e+00],
         [-4.0978e+00],
         [ 6.7342e+00],
         [ 7.2993e+00],
         [ 4.7352e+00],
         [ 2.6201e+00],
         [ 1.2321e+00],
         [ 3.1221e+00],
         [ 4.4439e+00],
         [ 5.1896e+00],
         [ 9.0921e+00],
         [ 5.5351e+00],
         [-9.0679e-01],
         [ 1.0640e+00],
         [ 1.1819e+01],
         [-3.4080e-01],
         [ 3.7855e+00],
         [ 1.2105e+01],
         [ 8.2630e+00],
         [-4.2923e-01],
         [-1.4404e+00],
         [ 6.6066e+00],
         [-1.7162e+00],
         [ 4.5325e

In [11]:
### 读取数据集


def data_iter(batch_size, features, labels):
    num_examples = len(features)
    indices = list(range(num_examples))
    # 这些样本是随机读取的，没有特定的顺序
    random.shuffle(indices)
    for i in range(0, num_examples, batch_size):
        batch_indices = torch.tensor(
            indices[i: min(i + batch_size, num_examples)])
        yield features[batch_indices], labels[batch_indices]

batch_size = 10

for X, y in data_iter(batch_size, features, labels):
    print(X, '\n', y)
    break

tensor([[-0.0861,  0.0628],
        [ 0.0643,  0.5139],
        [-0.0751,  0.7830],
        [ 0.4196, -0.4218],
        [ 0.1508, -1.6896],
        [-0.1250, -0.5861],
        [-0.7570, -0.1297],
        [ 4.1190, -0.1138],
        [-0.7229, -0.3896],
        [ 0.3103, -1.4336]]) 
 tensor([[ 3.8175],
        [ 2.5625],
        [ 1.3803],
        [ 6.4755],
        [10.2528],
        [ 5.9604],
        [ 3.1367],
        [12.8271],
        [ 4.0895],
        [ 9.7106]])


In [13]:
### 模型参数初始化

w = torch.normal(0, 0.01, size=(2,1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)


## 定义模型
def linreg(X, w, b):  #@save
    """线性回归模型"""
    return torch.matmul(X, w) + b

## 损失函数
def squared_loss(y_hat, y):  #@save
    """均方损失"""
    return (y_hat - y.reshape(y_hat.shape)) ** 2 / 2

## 优化算法：SGD
def sgd(params, lr, batch_size):  #@save
    """小批量随机梯度下降"""
    with torch.no_grad():
        for param in params:
            param -= lr * param.grad / batch_size
            param.grad.zero_()



In [14]:
## 训练

lr = 0.03
num_epochs = 3
net = linreg
loss = squared_loss

for epoch in range(num_epochs):
    for X, y in data_iter(batch_size, features, labels):
        l = loss(net(X, w, b), y)  # X和y的小批量损失
        # 因为l形状是(batch_size,1)，而不是一个标量。l中的所有元素被加到一起，
        # 并以此计算关于[w,b]的梯度
        l.sum().backward()
        sgd([w, b], lr, batch_size)  # 使用参数的梯度更新参数
    with torch.no_grad():
        train_l = loss(net(features, w, b), labels)
        print(f'epoch {epoch + 1}, loss {float(train_l.mean()):f}')

print(f'w的估计误差: {true_w - w.reshape(true_w.shape)}')
print(f'b的估计误差: {true_b - b}')


epoch 1, loss 0.035798
epoch 2, loss 0.000127
epoch 3, loss 0.000050
w的估计误差: tensor([ 0.0008, -0.0008], grad_fn=<SubBackward0>)
b的估计误差: tensor([0.0004], grad_fn=<RsubBackward1>)
